# Loop 1 LB Feedback Analysis

## Submission Results
- **CV Score**: 0.8361
- **LB Score**: 0.7799
- **Gap**: +0.0562 (CV is overly optimistic by ~5.6%)

## Key Questions
1. Why is CV so much higher than LB?
2. Is there distribution shift between train and test?
3. What can we do to close this gap?

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

# Load data
train = pd.read_csv('/home/data/train.csv')
test = pd.read_csv('/home/data/test.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

In [ ]:
# Compare distributions between train and test
print("=== Distribution Comparison ===")

print("\n1. Sex Distribution:")
print("Train:", train['Sex'].value_counts(normalize=True).to_dict())
print("Test:", test['Sex'].value_counts(normalize=True).to_dict())

print("\n2. Pclass Distribution:")
print("Train:", train['Pclass'].value_counts(normalize=True).sort_index().to_dict())
print("Test:", test['Pclass'].value_counts(normalize=True).sort_index().to_dict())

print("\n3. Age Statistics:")
print(f"Train: mean={train['Age'].mean():.1f}, std={train['Age'].std():.1f}")
print(f"Test: mean={test['Age'].mean():.1f}, std={test['Age'].std():.1f}")

print("\n4. Fare Statistics:")
print(f"Train: mean={train['Fare'].mean():.1f}, std={train['Fare'].std():.1f}")
print(f"Test: mean={test['Fare'].mean():.1f}, std={test['Fare'].std():.1f}")

In [ ]:
# Analyze the gap more deeply
# The CV-LB gap of 0.056 is significant
# This suggests either:
# 1. Overfitting to training data patterns
# 2. Distribution shift between train and test
# 3. Features that work well on train but not test

# Let's check the actual survival rate we'd expect
# If test has similar distribution to train, expected survival should be ~38%

# Our model's predictions on test
# From the submission, we can check what we predicted
submission = pd.read_csv('/home/code/submission_candidates/candidate_000.csv')
print(f"Predicted survival rate in test: {submission['Survived'].mean():.3f}")
print(f"Training survival rate: {train['Survived'].mean():.3f}")

In [ ]:
# Let's analyze what the LB score of 0.7799 means
# 0.7799 accuracy on 418 test samples = ~326 correct predictions
# 0.8361 CV accuracy on 891 train samples = ~745 correct predictions (in CV)

lb_score = 0.7799
cv_score = 0.8361
test_size = 418
train_size = 891

print(f"LB: {lb_score:.4f} accuracy = ~{int(lb_score * test_size)} correct out of {test_size}")
print(f"CV: {cv_score:.4f} accuracy = ~{int(cv_score * train_size)} correct out of {train_size}")
print(f"\nGap: {cv_score - lb_score:.4f} = {(cv_score - lb_score) * 100:.1f}% points")
print(f"\nThis gap is LARGE for Titanic. Typical gaps are 0.01-0.03.")
print(f"A gap of 0.056 suggests significant overfitting or distribution shift.")

In [ ]:
# Let's check if there's a pattern in what we might be getting wrong
# Key insight: The test set might have different characteristics

# Check Embarked distribution
print("Embarked Distribution:")
print("Train:", train['Embarked'].value_counts(normalize=True).to_dict())
print("Test:", test['Embarked'].value_counts(normalize=True).to_dict())

# Check SibSp/Parch distribution
print("\nSibSp Distribution:")
print("Train:", train['SibSp'].value_counts(normalize=True).sort_index().head().to_dict())
print("Test:", test['SibSp'].value_counts(normalize=True).sort_index().head().to_dict())

print("\nParch Distribution:")
print("Train:", train['Parch'].value_counts(normalize=True).sort_index().head().to_dict())
print("Test:", test['Parch'].value_counts(normalize=True).sort_index().head().to_dict())

In [ ]:
# Extract Title and compare
import re

def extract_title(name):
    match = re.search(r' ([A-Za-z]+)\.', name)
    if match:
        return match.group(1)
    return 'Unknown'

train['Title'] = train['Name'].apply(extract_title)
test['Title'] = test['Name'].apply(extract_title)

# Group rare titles
title_mapping = {
    'Mr': 'Mr', 'Miss': 'Miss', 'Mrs': 'Mrs', 'Master': 'Master',
    'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs',
    'Dr': 'Rare', 'Rev': 'Rare', 'Col': 'Rare', 'Major': 'Rare', 'Capt': 'Rare',
    'Lady': 'Rare', 'Countess': 'Rare', 'Sir': 'Rare', 'Don': 'Rare', 'Dona': 'Rare',
    'Jonkheer': 'Rare'
}
train['Title'] = train['Title'].map(lambda x: title_mapping.get(x, 'Rare'))
test['Title'] = test['Title'].map(lambda x: title_mapping.get(x, 'Rare'))

print("Title Distribution:")
print("Train:", train['Title'].value_counts(normalize=True).to_dict())
print("Test:", test['Title'].value_counts(normalize=True).to_dict())

In [ ]:
# Key insight: The distributions look similar
# The gap is likely due to:
# 1. Random variation in small test set (418 samples)
# 2. Model overfitting to training patterns
# 3. Some edge cases in test that don't match training patterns

# Let's check the gender submission baseline
gender_sub = pd.read_csv('/home/data/gender_submission.csv')
print(f"Gender submission survival rate: {gender_sub['Survived'].mean():.3f}")
print(f"This is the 'all females survive' baseline")

# Compare with our predictions
print(f"\nOur predicted survival rate: {submission['Survived'].mean():.3f}")
print(f"Difference: {submission['Survived'].mean() - gender_sub['Survived'].mean():.3f}")

## Analysis Summary

### CV-LB Gap Interpretation
- **Gap of +0.056 is significant** - suggests our model is overfitting to training patterns
- The distributions between train and test look similar, so it's not obvious distribution shift
- The gap could be due to:
  1. Model complexity (RF with depth 6 might be overfitting)
  2. Features that capture training-specific patterns
  3. Random variation in the small test set

### Implications for Strategy
1. **Simpler models might generalize better** - consider reducing complexity
2. **Ensemble diversity is key** - different models might capture different patterns
3. **Feature selection** - some features might be overfitting
4. **The target of 1.0 is still unrealistic** - best legitimate scores are ~0.80-0.82

### Next Steps
1. Try simpler models (Logistic Regression, shallow trees)
2. Implement ensemble with diverse models
3. Add ticket-based features (might help with group survival patterns)
4. Consider regularization to reduce overfitting